In [ ]:
# default_exp Tutorial

# Getting Started Tutorial with Tabular ML Toolkit

> A tutorial on getting started with ML Toolkit, a helper library to jumpstart your machine learning project based on Tabular or Structured data.

## Install

`pip install -U tabular_ml_toolkit`

## How to Best Use tabular_ml_toolkit

Start with your favorite model and then just simply create MLPipeline with one API.

*For example, Here we are using RandomForestRegressor from Scikit-Learn, on  [Melbourne Home Sale price data](https://www.kaggle.com/estrotococo/home-data-for-ml-course)*


*No need to install scikit-learn as it comes preinstall with Tabular_ML_Toolkit*

In [ ]:
#export
from tabular_ml_toolkit.MLPipeline import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np

# for displaying diagram of pipelines 
from sklearn import set_config
set_config(display="diagram")

In [ ]:
#export
# Dataset file names and Paths
DIRECTORY_PATH = "https://raw.githubusercontent.com/psmathur/tabular_ml_toolkit/master/input/home_data/"
TRAIN_FILE = "train.csv"
TEST_FILE = "test.csv"
SAMPLE_SUB_FILE = "sample_submission.csv"

In [ ]:
#export
# create scikit-learn ml model
scikit_model = RandomForestRegressor(n_estimators=200, random_state=42)

# createm ml pipeline for scikit-learn model
ml_pl = MLPipeline().prepare_data_for_training(
    train_file_path= DIRECTORY_PATH+TRAIN_FILE,
    test_file_path= DIRECTORY_PATH+TEST_FILE,
    idx_col="Id", target="SalePrice",
    model=scikit_model,
    random_state=42)

In [ ]:
ml_pl.spl

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'Lo...
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...]),
                                                 ('high_card_cat_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Neighborhood',
                                                   'Exterior1st',
                                                   'Exterior2nd'])])),
                ('model',
                 RandomForestRegressor(n_estimators=200, random_state=42))])

In [ ]:
ml_pl.dfl.create_train_valid(valid_size=0.2)

In [ ]:
# # Now fit and predict
ml_pl.spl.fit(ml_pl.dfl.X_train, ml_pl.dfl.y_train)

preds = ml_pl.spl.predict(ml_pl.dfl.X_valid)
print('X_valid MAE:', mean_absolute_error(ml_pl.dfl.y_valid, preds))

X_valid MAE: 17483.238390410956


#### Let's do Cross Validation for Scikit-Learn Model on our MLPipeline

In [ ]:
# Now fit and predict
scores = ml_pl.do_cross_validation(cv=3, scoring='neg_mean_absolute_error')
print("scores:", scores)
print("Average MAE score:", scores.mean())

scores: [17701.71892197 17568.5187269  18495.24910494]
Average MAE score: 17921.828917936306


In background `prepare_data_for_training` and `prepare_data_for_cv`  methods loads your input data into Pandas DataFrame, seprates X(features) and y(target).

Then `prepare_data_for_training` methods split X(features) into X_train, y_train, X_valid and y_valid DataFrames.
However, `prepare_data_for_cv`  method do not split but let's cross validation split internally X and y DataFrames.

Then both methods preprocess all numerical and categorical type data found in these DataFrames using scikit-learn pipelines. Then it bundle preprocessed data with your given model and return an MLPipeline object, this class instance has dataframeloader, preprocessor and scikit-lean pipeline instances, so you can call fit methods on X_train and y_train and predict methods on X_valid or X_test.

Please check detail documentation and source code for more details.

*NOTE: If you want to customize data and preprocessing steps you can do so by using `DataFrameLoader` and `PreProessor` classes. Check detail documentations for these classes for more options.*

#### Let's do Grid Search for HyperParameters Tunning for Scikit Model on our MLPipeline

In [ ]:
# param_grid = {
# #     "preprocessor__num__imputer__strategy": ["constant", "mean", "median"],
# #     "preprocessor__low_cad_cat__imputer__strategy": ["most_frequent", "constant"],
# #     "model__max_depth": [40,80,100],
# #     'model__max_features': [2, 3],
# #     'model__min_samples_leaf': [3,4,5],
# #     'model__min_samples_split': [8,10,12],
#     "model__n_estimators": [250,550]
# }

# grid_search = ml_pl.do_grid_search(param_grid=param_grid, cv=3,
#                                        scoring='neg_mean_absolute_error')

# print("Best params:")
# print(grid_search.best_params_)

# print(f"Internal CV Metrics score: {-1*(grid_search.best_score_):.3f}")

#### Let's Use K-Fold Training with best params from grid search

In [ ]:
# # create a new scikit-learn RandomForestRegressor model with best params from grid search
# scikit_model_new = RandomForestRegressor(
#     n_estimators=550,
# #     max_depth = 80,
# #     max_features = 3,
# #     min_samples_leaf = 3,
# #     min_samples_split = 8,
#     random_state=42)

In [ ]:
# # Update pipeline with new model
# ml_pl.model = scikit_model_new

In [ ]:
# ml_pl.spl

In [ ]:
# print(len(sci_ml_pl.dfl.final_cols))
# sci_ml_pl.dataframeloader.final_cols

In [ ]:
# k-fold training
k_fold, metrics_score = ml_pl.do_k_fold_training(n_splits=5, metrics=mean_absolute_error)
print("mean metrics score:", np.mean(metrics_score))

/Users/pamathur/miniconda3/envs/nbdev_env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


fold: 1 , mean_absolute_error: 18568.04107876712
fold: 2 , mean_absolute_error: 16109.12285958904
fold: 3 , mean_absolute_error: 18086.338938356163
fold: 4 , mean_absolute_error: 16517.901232876713
fold: 5 , mean_absolute_error: 17584.88452054795
mean metrics score: 17373.257726027397


In [ ]:
# predict
preds = ml_pl.do_k_fold_prediction(k_fold=k_fold)
print(preds.shape)
# preds

(1459,)


#### Let's Use XGBosst on MLPipeline

*You can also use MLPipeline with XGBoost model, Just make sure to install XGBooost first depending upon your OS.*

*After that all steps remains same. Here is example using XGBRegressor with [Melbourne Home Sale price data](https://www.kaggle.com/estrotococo/home-data-for-ml-course)*

In [ ]:
#!pip install -U xgboost

In [ ]:
# Best way to install xgboost if you are on macosx and windows machine is using conda
# !conda install -c conda-forge xgboost

In [ ]:
#export
from xgboost import XGBRegressor
# create xgb ml model
xgb_model = XGBRegressor(n_estimators=250,learning_rate=0.05, random_state=42)

In [ ]:
# Update pipeline with xgb model
ml_pl.model = xgb_model

In [ ]:
# Now fit and predict
ml_pl.spl.fit(ml_pl.dfl.X_train, ml_pl.dfl.y_train)
preds = ml_pl.spl.predict(ml_pl.dfl.X_valid)
print('X_valid MAE:', mean_absolute_error(ml_pl.dfl.y_valid, preds))

X_valid MAE: 17584.88452054795


#### Let's do Cross Validation for XGB Model on our MLPipeline

In [ ]:
# cross_validation
scores = ml_pl.do_cross_validation(cv=3, scoring='neg_mean_absolute_error')
print("scores:", scores)
print("Average MAE score:", scores.mean())

scores: [17701.71892197 17568.5187269  18495.24910494]
Average MAE score: 17921.828917936306


#### Let's Use K-Fold Training for XGB model

In [ ]:
# k-fold training
k_fold, metrics_score = ml_pl.do_k_fold_training(n_splits=5, metrics=mean_absolute_error)
print("mean metrics score:", np.mean(metrics_score))

/Users/pamathur/miniconda3/envs/nbdev_env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


fold: 1 , mean_absolute_error: 18568.04107876712
fold: 2 , mean_absolute_error: 16109.12285958904
fold: 3 , mean_absolute_error: 18086.338938356163
fold: 4 , mean_absolute_error: 16517.901232876713
fold: 5 , mean_absolute_error: 17584.88452054795
mean metrics score: 17373.257726027397


In [ ]:
# predict
preds = ml_pl.do_k_fold_prediction(k_fold=k_fold)
print(preds.shape)
# preds

(1459,)


In [ ]:
# hide
# run the script to build 

from nbdev.export import notebook2script; notebook2script()

Converted 00_DataFrameLoader.ipynb.
Converted 01_PreProcessor.ipynb.
Converted 02_MLPipeline.ipynb.
Converted 03_Tutorial.ipynb.
Converted 07_Kaggle_TPS_Tutorial.ipynb.
Converted Optuna_Tutorial.ipynb.
Converted automl_in_sklearn_pipeline.ipynb.
Converted index.ipynb.
